# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [11]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kloen\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

In [13]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key = api_key)

In [14]:
# Fetch the Bitcoin news articles
bitcoin = newsapi.get_everything(q = 'bitcoin', language = 'en', sort_by = 'relevancy')

In [15]:
# Fetch the Ethereum news articles
ethereum = newsapi.get_everything(q = 'ethereum', language = 'en', sort_by = 'relevancy')

In [16]:
print(ethereum['articles'][0]['content'])

A new cross-chain bridge is currently connected to Ethereum through a cross-chain bridge, with Cardano and other public chains to come in the future.
Nervos today announced that the Force Bridge is … [+3114 chars]


In [5]:
# Create the Bitcoin sentiment scores DataFrame
sentiments = []

for articles in bitcoin["articles"]:
    try:
        text = articles["content"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "text": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
btc  = pd.DataFrame(sentiments)
btc.head()

NameError: name 'bitcoin' is not defined

In [18]:
# Create the Ethereum sentiment scores DataFrame
sentiments = []

for articles in ethereum["articles"]:
    try:
        text = articles["content"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "text": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
eth  = pd.DataFrame(sentiments)
eth.head()

,text,Compound,Positive,Negative,Neutral
0,A new cross-chain bridge is currently connecte...,0.0000,0.000,0.0,1.000
1,Mark Cuban has some advice for people who are ...,0.0000,0.000,0.0,1.000
2,"In keeping with a previous announcement, AMC t...",0.5719,0.153,0.0,0.847
3,Ethereum and bitcoin are the two biggest crypt...,0.4588,0.094,0.0,0.906
4,Elon Musk\r\npicture alliance / Getty Images\r...,0.5267,0.093,0.0,0.907


In [19]:
# Describe the Bitcoin Sentiment
btc.describe()

,Compound,Positive,Negative,Neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.220215,0.054550,0.003150,0.942300
std,0.292918,0.066502,0.014087,0.065262
min,-0.273200,0.000000,0.000000,0.801000
25%,0.000000,0.000000,0.000000,0.896500
50%,0.000000,0.000000,0.000000,0.968500
75%,0.481000,0.103500,0.000000,1.000000
max,0.755800,0.199000,0.063000,1.000000


In [20]:
# Describe the Ethereum Sentiment
eth.describe()

,Compound,Positive,Negative,Neutral
count,20.000000,20.000000,20.0,20.000000
mean,0.280475,0.063150,0.0,0.936850
std,0.308037,0.074661,0.0,0.074661
min,0.000000,0.000000,0.0,0.779000
25%,0.000000,0.000000,0.0,0.902500
50%,0.180600,0.036000,0.0,0.964000
75%,0.530600,0.097500,0.0,1.000000
max,0.822500,0.221000,0.0,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum has the higher mean score

Q: Which coin had the highest compound score?

A: Same compound score

Q. Which coin had the highest positive score?

A: Ethereum

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [21]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [22]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
stop_words = stopwords.words('english')

# Expand the default stopwords list if necessary
stop_words.append('new_word')

In [23]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""


    # Create a list of the words
    words = word_tokenize(text)

    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))
    
    # Remove the punctuation
    words = list(filter(lambda t: t not in punctuation, words))
    
    # Remove the stopwords
    words = list(filter(lambda t: t.lower() not in stop_words, words))
    
    # Lemmatize Words into root words
    tokens = [lemmatizer.lemmatize(word) for word in words]
    
    return tokens

In [24]:
# Create a new tokens column for Bitcoin
btc['tokens']= btc.text.apply(tokenizer)
btc.head()

,text,Compound,Positive,Negative,Neutral,tokens
0,A similar hoax earlier this year tied Walmart ...,-0.2732,0.000,0.063,0.937,"[similar, hoax, earlier, year, tied, Walmart, ..."
1,Theres a big new presence slurping up power fr...,0.3612,0.096,0.000,0.904,"[Theres, big, new, presence, slurping, power, ..."
2,"For all the talk of democratizing finance, the...",0.0000,0.000,0.000,1.000,"[talk, democratizing, finance, vast, majority,..."
3,"In keeping with a previous announcement, AMC t...",0.5719,0.153,0.000,0.847,"[keeping, previous, announcement, AMC, theater..."
4,Representation of cryptocurrency Bitcoin is pl...,0.0000,0.000,0.000,1.000,"[Representation, cryptocurrency, Bitcoin, plac..."


In [25]:
# Create a new tokens column for Ethereum
def tokenizer(text):
    """Tokenizes text."""


    # Create a list of the words
    words = word_tokenize(text)

    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))
    
    # Remove the punctuation
    words = list(filter(lambda t: t not in punctuation, words))
    
    # Remove the stopwords
    words = list(filter(lambda t: t.lower() not in stop_words, words))
    
    # Lemmatize Words into root words
    tokens = [lemmatizer.lemmatize(word) for word in words]
    
    return tokens

In [26]:
# Create a new tokens column for Bitcoin
eth['tokens']= btc.text.apply(tokenizer)
eth.head()

,text,Compound,Positive,Negative,Neutral,tokens
0,A new cross-chain bridge is currently connecte...,0.0000,0.000,0.0,1.000,"[similar, hoax, earlier, year, tied, Walmart, ..."
1,Mark Cuban has some advice for people who are ...,0.0000,0.000,0.0,1.000,"[Theres, big, new, presence, slurping, power, ..."
2,"In keeping with a previous announcement, AMC t...",0.5719,0.153,0.0,0.847,"[talk, democratizing, finance, vast, majority,..."
3,Ethereum and bitcoin are the two biggest crypt...,0.4588,0.094,0.0,0.906,"[keeping, previous, announcement, AMC, theater..."
4,Elon Musk\r\npicture alliance / Getty Images\r...,0.5267,0.093,0.0,0.907,"[Representation, cryptocurrency, Bitcoin, plac..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [27]:
from collections import Counter
from nltk import ngrams

In [28]:
# Generate the Bitcoin N-grams where N=2
N = 2
grams = ngrams(tokenizer(btc.text.str.cat()), N)
Counter(grams).most_common(20)

[(('Reuters', 'Bitcoin'), 5),
 (('cryptocurrency', 'Bitcoin'), 4),
 (('illustration', 'taken'), 4),
 (('Bitcoin', 'seen'), 4),
 (('exchange-traded', 'fund'), 4),
 (('char', 'Bitcoin'), 4),
 (('representation', 'virtual'), 3),
 (('virtual', 'cryptocurrency'), 3),
 (('seen', 'picture'), 3),
 (('picture', 'illustration'), 3),
 (('taken', 'October'), 3),
 (('2021', 'REUTERS/Edgar'), 3),
 (('Bitcoin', 'hit'), 3),
 (('KONG', 'Oct'), 2),
 (('char', 'representation'), 2),
 (('October', '19'), 2),
 (('19', '2021'), 2),
 (('Oct', '20'), 2),
 (('20', 'Reuters'), 2),
 (('char', 'Posted'), 2)]

In [29]:
# Generate the Ethereum N-grams where N=2
N = 2
grams = ngrams(tokenizer(eth.text.str.cat()), N)
Counter(grams).most_common(20)

[(('Bitcoin', 'Ethereum'), 5),
 (('illustration', 'taken'), 4),
 (('taken', 'June'), 4),
 (('Getty', 'Images'), 3),
 (('Rahul', 'Rai'), 3),
 (('cross-chain', 'bridge'), 2),
 (('Elon', 'Musk'), 2),
 (('New', 'York'), 2),
 (('char', 'Representations'), 2),
 (('Representations', 'cryptocurrencies'), 2),
 (('cryptocurrencies', 'Bitcoin'), 2),
 (('Ethereum', 'DogeCoin'), 2),
 (('DogeCoin', 'Ripple'), 2),
 (('Ripple', 'Litecoin'), 2),
 (('Litecoin', 'placed'), 2),
 (('placed', 'PC'), 2),
 (('PC', 'motherboard'), 2),
 (('motherboard', 'illustration'), 2),
 (('June', '29'), 2),
 (('29', '2021'), 2)]

In [30]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [31]:
# Use token_count to get the top 10 words for Bitcoin
all_tokens = tokenizer(btc.text.str.cat())
token_count(all_tokens, 10)

[('Bitcoin', 20),
 ('char', 19),
 ('U.S.', 6),
 ('bitcoin', 6),
 ('Reuters', 6),
 ('cryptocurrency', 5),
 ('taken', 5),
 ('2021', 5),
 ('record', 5),
 ('ETF', 5)]

In [32]:
# Use token_count to get the top 10 words for Ethereum
all_tokens = tokenizer(eth.text.str.cat())
token_count(all_tokens, 10)

[('char', 20),
 ('Ethereum', 11),
 ('cryptocurrency', 8),
 ('Bitcoin', 8),
 ('cryptocurrencies', 6),
 ('…', 5),
 ('bitcoin', 4),
 ('Images', 4),
 ('illustration', 4),
 ('taken', 4)]

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [3]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [4]:
# Generate the Bitcoin word cloud
btc_wc = WordCloud().generate(' '.join(btc))
plt.imshow(btc_wc)

NameError: name 'btc' is not defined

In [2]:
# Generate the Ethereum word cloud
eth_wc = WordCloud().generate(' '.join(eth))
plt.imshow(eth_wc)

NameError: name 'WordCloud' is not defined

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm


In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
all_btc_text = btc.text.str.cat()
all_btc_text

In [ ]:

# YOUR CODE HERE!
# Run the NER processor on all of the text
doc = nlp(all_btc_text)
# Add a title to the document
doc.user_data['title'] = "Bitcoin NER"

In [ ]:
# Render the visualization
# Render the visualization
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# List all Entities
for ent in doc.ents:
    print(ent.text, ent.label_)

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
all_eth_text = eth.text.str.cat()
all_eth_text

In [ ]:
# Run the NER processor on all of the text
doc = nlp(all_eth_text)
# Add a title to the document
doc.user_data['title'] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# List all Entities
for ent in doc.ents:
    print(ent.text, ent.label_)

---